In [236]:
!pip install python-dotenv -q
!pip install openai -q
!pip install --upgrade langchain -q

In [230]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
load_dotenv(find_dotenv())
OPENAI_API_KEY="[your API key]"


# Basic OpenAI call

In [231]:
import os
from openai import OpenAI
#client = OpenAI(openai.api_key =OPENAI_API_KEY)
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=OPENAI_API_KEY,
)



In [16]:
llm_model="gpt-3.5-turbo-0301"
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content
def get_completion_full(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response

In [13]:
get_completion("where does term `underdog` come from?")

'The term "underdog" originated in the world of dogfighting in the 19th century. The weaker or less aggressive dog was placed in a pit with a stronger or more aggressive dog, and the weaker dog was referred to as the "underdog." The term later became popularized in sports and other competitive contexts to refer to a person or team that is expected to lose or is at a disadvantage.'

## Building Simple prompt

In [14]:
instruction="your are expert in history. your role is to give answer with lowest token but accurate"
question="why david was called the underdog?"

prompt=f"""{instruction}. Answer the following question:
-----
question: {question}
"""

In [15]:
get_completion(prompt)

'David was called the underdog because he was a young and inexperienced shepherd who defeated the giant Goliath in battle.'

In [37]:
get_completion_full(prompt)

ChatCompletion(id='chatcmpl-8qmgoJLdSQDlUhpApStvIrGhfgWG0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='David was called the underdog because he was a young and inexperienced shepherd who defeated the giant Goliath in battle.', role='assistant', function_call=None, tool_calls=None))], created=1707590910, model='gpt-3.5-turbo-0301', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=42, total_tokens=66))

In [42]:
#obj=["id","choices", "model", "usage"]
print(get_completion_full(prompt).id)
print("-"*50)
print(get_completion_full(prompt).choices)
print("-"*50)
print(get_completion_full(prompt).model)
print("-"*50)
print(get_completion_full(prompt).usage)

chatcmpl-8qmk6PR2wBUqHQCjAKq1CnpJxc4ZK
--------------------------------------------------
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='David was called the underdog because he was a young shepherd who defeated the giant Goliath in battle.', role='assistant', function_call=None, tool_calls=None))]
--------------------------------------------------
gpt-3.5-turbo-0301
--------------------------------------------------
CompletionUsage(completion_tokens=24, prompt_tokens=42, total_tokens=66)


#OpenAI API with LangChain

In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [66]:
chat=ChatOpenAI(temperature=0,model=llm_model, openai_api_key=OPENAI_API_KEY)

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

tmp=ChatPromptTemplate.from_template(template_string)
prompt=tmp.format_messages(style=customer_style,text=customer_email)
respond=chat(prompt)
respond.content

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"

In [ ]:
### alternstive witj LLMChain
from langchain.chains import LLMChain
llm = ChatOpenAI(temperature=0.9, model=llm_model)
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "cooling system for ion traps"
chain.run(product)

In [ ]:
## another method
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

qdocs="[some text]"
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")


## output Schema and Parser

In [71]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [80]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

### this doesnt work, it resturn str. which is not helpful
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [74]:
prompt_template=ChatPromptTemplate.from_template(review_template)
prompt=prompt_template.format_messages(text=customer_review)



In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [93]:
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')])

In [94]:
format_instructions=output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.\n\t"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [83]:
prompt=ChatPromptTemplate.from_template(review_template_2)
prompt=prompt.format_messages(text=customer_review,
                              format_instructions=format_instructions)

In [86]:
response=chat(prompt)

In [87]:
response.content

'```json\n{\n\t"gift": true,\n\t"delivery_days": "2",\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'

In [92]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

# Memory

## ConversationBufferMemory

In [98]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [102]:
llm=ChatOpenAI(temperature=0, model=llm_model,openai_api_key=OPENAI_API_KEY)

In [103]:
memory=ConversationBufferMemory()

In [105]:
conversation=ConversationChain(llm=llm, memory=memory,verbose=True)

In [106]:
conversation.predict(input="Hi, my name is Ali")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ali
AI:

> Finished chain.


"Hello Ali, it's nice to meet you! My name is AI. How can I assist you today?"

In [107]:
conversation.predict(input="what is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ali
AI: Hello Ali, it's nice to meet you! My name is AI. How can I assist you today?
Human: what is my name?
AI:

> Finished chain.


'Your name is Ali, as you mentioned earlier. Is there anything else I can help you with?'

In [109]:
memory.buffer

"Human: Hi, my name is Ali\nAI: Hello Ali, it's nice to meet you! My name is AI. How can I assist you today?\nHuman: what is my name?\nAI: Your name is Ali, as you mentioned earlier. Is there anything else I can help you with?"

In [110]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Ali\nAI: Hello Ali, it's nice to meet you! My name is AI. How can I assist you today?\nHuman: what is my name?\nAI: Your name is Ali, as you mentioned earlier. Is there anything else I can help you with?"}

In [111]:
memory = ConversationBufferMemory()

In [114]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [115]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Hi\nAI: What's up\nHuman: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [116]:
from langchain.memory import ConversationBufferWindowMemory

In [122]:
memory=ConversationBufferWindowMemory(k=2)

In [123]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "what is the purpose of life?"},
                    {"output": "Nothing ^.^"})

In [124]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool\nHuman: what is the purpose of life?\nAI: Nothing ^.^'}

In [127]:
llm=ChatOpenAI(temperature=0, model=llm_model,openai_api_key=OPENAI_API_KEY)
#memory=ConversationBufferMemory()
memory=ConversationBufferWindowMemory(k=2)
conversation=ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

## ConversationTokenBufferMemory

In [128]:
from langchain.memory import ConversationTokenBufferMemory

In [129]:
memory=ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50
    )

## ConversationSummaryMemory

In [130]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# Chain

In [132]:
import pandas as pd
df =pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Product').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Review').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Review'].value_counts()
    for x_label, grp in _df_2.groupby('Product')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Product')
_ = plt.ylabel('Review')

In [133]:
from langchain.chains import LLMChain

In [135]:
llm=ChatOpenAI(temperature=0.9, model=llm_model,openai_api_key=OPENAI_API_KEY)

In [154]:
prompt=ChatPromptTemplate.from_template(
    "What is the best name for thesis to which \
    covering the following topics at each chapters? try to be abit creative: {chapters}"
)

chapters= """
1-Double Descent in Quantum Learning Task
2-Surviving The Barren Plateau in Variational Quantum Circuits
with Bayesian Learning Initialization
3-Deep Quantum Neural Networks are Gaussian Process
4-Efficient Hamiltonian Calibration Through Pruning Control Parameters
5-Simulating a quantum lattice field theory with donor spins in silicon
6-Recovering Superfast Encoding Using The Underlying Lattice Model
"""
chain=LLMChain(llm=llm, prompt=prompt)

In [155]:
chain.run(chapters)

'"Quantum Leap: Exploring Novel Techniques in Quantum Machine Learning and Simulation"'

## SimpleSequentialChain

In [156]:
from langchain.chains import SimpleSequentialChain

In [165]:
first_prompt=ChatPromptTemplate.from_template(
    "What is the best name for thesis to which \
    covering the following topics at each chapters? try to be abit creative: {chapters}"
)

chapters= """
1-Double Descent in Quantum Learning Task
2-Surviving The Barren Plateau in Variational Quantum Circuits
with Bayesian Learning Initialization
3-Deep Quantum Neural Networks are Gaussian Process
4-Efficient Hamiltonian Calibration Through Pruning Control Parameters
5-Simulating a quantum lattice field theory with donor spins in silicon
6-Recovering Superfast Encoding Using The Underlying Lattice Model
"""
chain_1=LLMChain(llm=llm, prompt=first_prompt)

second_prompt=ChatPromptTemplate.from_template(" write a 20 word decription for the following thesisis tile: {title }")
chain_2=LLMChain(llm=llm,prompt=second_prompt)

In [166]:
simple_chain=SimpleSequentialChain(chains=[chain_1, chain_2],
                                             verbose=True
                                            )

In [167]:
simple_chain.run(chapters)



> Entering new SimpleSequentialChain chain...
Quantum Quest: Navigating the Frontiers of Quantum Machine Learning, Calibration, and Simulation
This thesis explores the emerging field of quantum machine learning, calibration, and simulation to unlock the potential of quantum technologies.

> Finished chain.


'This thesis explores the emerging field of quantum machine learning, calibration, and simulation to unlock the potential of quantum technologies.'

##SequentialChain

In [168]:
from langchain.chains import SequentialChain

In [170]:
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=OPENAI_API_KEY)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)


In [171]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t last, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'summary': 'The reviewer found the taste of the product to be mediocre and suspects that it might be an old batch or counterfeit due to the lack of foam.',
 'followup_message': "Le critique a trouvé le goût du produit moyen et soupçonne qu'il pourrait s'agir d'un lot ancien ou contrefait en raison du manque de mousse."}

# Routing Chain

In [173]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [180]:
## list
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [183]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [187]:
llm=ChatOpenAI( temperature=0,model=llm_model,openai_api_key=OPENAI_API_KEY)

In [212]:
destination_chains={}
destinations=[]
for p_info in prompt_infos: ## calling item from list
    # each item is a dict
    destinations.append(f'{p_info["name"]}:{p_info["description"]}')
    name=p_info["name"]
    prompt_template=p_info["prompt_template"]
    prompt=ChatPromptTemplate.from_template(template=prompt_template)
    chain=LLMChain(llm=llm, prompt=prompt)
    destination_chains[name]=chain

destinations_str = "\n".join(destinations)


In [209]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [210]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [214]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str)

In [215]:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics:Good for answering questions about physics\nmath:Good for answering math que

In [219]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [221]:
router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [222]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [223]:
chain.run("What is AdS/CFT?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is the AdS/CFT correspondence?'}
> Finished chain.


'The AdS/CFT correspondence is a theoretical framework in physics that relates two seemingly different theories: Anti-de Sitter space (AdS) and conformal field theory (CFT). It suggests that there is a duality between a gravitational theory in AdS space and a quantum field theory on its boundary, which is a conformal field theory. This correspondence has been used to study various phenomena in both gravitational and quantum field theories, and has led to new insights in areas such as black hole physics and the nature of quantum gravity.'

In [224]:
chain.run("Who was david and what was he famous for?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


History: {'input': 'Who was david and what was he famous for?'}
> Finished chain.


'David was a biblical figure and the second king of Israel. He is famous for defeating the giant Goliath with a slingshot and for establishing Jerusalem as the capital of Israel. He is also known for his affair with Bathsheba and for being the father of King Solomon. David is considered a significant figure in Jewish, Christian, and Islamic traditions.'

In [226]:
chain.run("What is Solomon paradox?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'What is Solomon paradox?'}
> Finished chain.


'The Solomon paradox refers to the phenomenon where people are better at giving advice to others than they are at applying the same advice to themselves. It is named after King Solomon, who was known for his wisdom and ability to give sound advice to others, but struggled to apply the same wisdom to his own life. The paradox highlights the discrepancy between our ability to think objectively and rationally when advising others, and our tendency to be biased and emotional when making decisions for ourselves.'

# Retriver from Documents

In [266]:
!pip install docarray -q
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [245]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
from langchain.indexes import VectorstoreIndexCreator

In [243]:
file='OutdoorClothingCatalog_1000.csv'
loader=CSVLoader(file_path=file)

In [254]:
llm_model = OpenAI(temperature=0,
                               model='gpt-3.5-turbo-instruct',
                   openai_api_key=OPENAI_API_KEY)

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [267]:
index=VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [272]:
my_query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."


In [274]:
#response=index.query(my_query, llm=llm_model)

In [275]:
embed = embeddings.embed_query("Hi my name is Ali")

In [276]:
print(len(embed))

1536


In [277]:
docs = loader.load()
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

In [278]:
query = "Please suggest a shirt with sunblocking"

In [280]:
#docs = db.similarity_search(query)

In [281]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")
display(Markdown(response))

In [282]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [283]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [285]:
#response = qa_stuff.run(query)
#display(Markdown(response))

In [286]:
## or simple method:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

# Evaluation

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch


file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)


examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

examples += new_examples
qa.run(examples[0]["query"])

import langchain
langchain.debug = True

qa.run(examples[0]["query"])

# Turn off the debug mode
langchain.debug = False


predictions = qa.apply(examples)

from langchain.evaluation.qa import QAEvalChain

llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(examples, predictions)

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()









many thanks for deeplearning.ai short courses to cover most of the detials and codes